<h1 align= 'center' style="font-family:Lucida Bright"><font size = 6>Foursqare Api </font></h1>

### *Group members*:
>##### Hoda Esfahani
>
>##### Md. Nazmul Haque Khan


In [1]:
# load necessary
import requests  # library to handle requests
import pandas as pd  # library for data analsysis
import numpy as np  # library to handle data in a vectorized manner
from pandas import json_normalize  # tranforming json file into a pandas dataframe library
import pickle  # handle pickle format data
import folium

# Run me if you'd like to install dataprep
#!pip install dataprep

In [2]:
# load plotly
import plotly.offline as py
import plotly.express as px
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from plotly import tools
pd.options.plotting.backend = "plotly"


In [3]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
# display multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
from IPython.display import Markdown, display


# display or print markdown as output
def printMd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))


def printFullDf(df):
    # set options temporarily to display one large DataFrame
    with pd.option_context('display.max_rows', None, 'display.max_columns',
                           None):
        display(df)

## Load Data

In [6]:
with open("dataframe_raw", "rb") as file:
    fs_df = pickle.load(file)

fs_df.head()

,id,name,canonicalUrl,categories,verified,url,like,dislike,ok,rating,...,page.user.contact.twitter,page.pageInfo.description,hasMenu,hours.seasonalRange.startMonth,hours.seasonalRange.startDay,hours.seasonalRange.endMonth,hours.seasonalRange.endDay,hours.seasonalRange.closed,hours.seasonalRange.displayDateRange,parent.venuePage.id
0,4bcc34bc937ca5937cafaa92,Lenininpuisto,https://foursquare.com/v/lenininpuisto/4bcc34b...,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,https://fi.wikipedia.org/wiki/Lenininpuisto,False,False,False,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5cf5c8c96fd626002c4fbf65,Konepaja Biergarten,https://foursquare.com/v/konepaja-biergarten/5...,"[{'id': '4bf58dd8d48988d117941735', 'name': 'B...",False,NaN,False,False,False,8.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b1809a0f964a520dccb23e3,Maltainen Riekko,https://foursquare.com/v/maltainen-riekko/4b18...,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",True,NaN,False,False,False,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55f32b4d498ed2c829de7b7d,Alppilan Huone,https://foursquare.com/v/alppilan-huone/55f32b...,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",False,http://www.alppilanhuone.fi,False,False,False,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5a240d39da5ede3855f51f73,Going Greek,https://foursquare.com/v/going-greek/5a240d39d...,"[{'id': '4bf58dd8d48988d10e941735', 'name': 'G...",False,https://www.goinggreek.fi,False,False,False,8.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
test = fs_df.loc[:, [
    'name', 'categories', 'location.city', 'location.country', 'location.lat',
    'location.lng', 'likes.count', 'rating', 'ratingSignals', 'ratingColor',
    'contact.phone', 'contact.formattedPhone', 'contact.twitter',
    'contact.facebook', 'contact.facebookUsername', 'contact.instagram',
    'contact.facebookName', 'stats.tipCount', 'photos.count', 'listed.count',
    'url'
]]
contact = test.loc[:, [
    'contact.phone', 'contact.formattedPhone', 'contact.twitter',
    'contact.facebook', 'contact.facebookUsername', 'contact.instagram',
    'contact.facebookName'
]]
test['has_contact'] = ~(contact.isna().all(axis=1))
test.drop([
    'contact.phone', 'contact.formattedPhone', 'contact.twitter',
    'contact.facebook', 'contact.facebookUsername', 'contact.instagram',
    'contact.facebookName'
],
          axis=1,
          inplace=True)
test['url'] = test['url'].isna()
test['categories'] = test['categories'].astype(str).str.extract(
    '(?<=\'name\': \')(.*?)(?=\', \'p)', expand=True)

In [8]:
test.columns = [
    'name', 'category', 'city', 'country', 'lat', 'lon', 'likes', 'rating',
    'rating_count', 'rating_color', 'tip_count', 'photos_count',
    'listed_count', 'has_website', 'has_contact'
]
fs_df = test
fs_df.tail()

,name,category,city,country,lat,lon,likes,rating,rating_count,rating_color,tip_count,photos_count,listed_count,has_website,has_contact
473,Restaurant Schützengasse,Swiss Restaurant,Zürich,Schweiz,47.377154,8.537430,36,8.5,51,73CF42,12,27,15,True,True
474,Zurich Main Station (Zürich Hauptbahnhof),Train Station,Zürich,Schweiz,47.378705,8.538724,1267,8.3,1643,73CF42,235,4746,222,False,True
475,H&M,Clothing Store,Zürich,Schweiz,47.375877,8.539477,28,7.9,36,C5DE35,1,26,9,False,True
476,Café Gourmet,Café,Zürich,Schweiz,47.377094,8.539425,8,7.9,11,C5DE35,6,14,2,False,True
477,Victorinox Flagship Store Zürich,Watch Shop,Zürich,Schweiz,47.374041,8.538962,39,8.7,46,73CF42,6,31,13,False,True


In [9]:
fs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          478 non-null    object 
 1   category      478 non-null    object 
 2   city          459 non-null    object 
 3   country       478 non-null    object 
 4   lat           478 non-null    float64
 5   lon           478 non-null    float64
 6   likes         478 non-null    int64  
 7   rating        478 non-null    float64
 8   rating_count  478 non-null    int64  
 9   rating_color  478 non-null    object 
 10  tip_count     478 non-null    int64  
 11  photos_count  478 non-null    int64  
 12  listed_count  478 non-null    int64  
 13  has_website   478 non-null    bool   
 14  has_contact   478 non-null    bool   
dtypes: bool(2), float64(3), int64(5), object(5)
memory usage: 49.6+ KB


In [10]:
fs_df.describe(include='all')

,name,category,city,country,lat,lon,likes,rating,rating_count,rating_color,tip_count,photos_count,listed_count,has_website,has_contact
count,478,478,459,478,478.000000,478.000000,478.000000,478.000000,478.000000,478,478.000000,478.000000,478.000000,478,478
unique,473,156,14,10,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,2,2
top,Amorino,Hotel,Paris,Magyarország,NaN,NaN,NaN,NaN,NaN,73CF42,NaN,NaN,NaN,False,True
freq,4,36,50,50,NaN,NaN,NaN,NaN,NaN,338,NaN,NaN,NaN,315,349
mean,NaN,NaN,NaN,NaN,48.999710,10.606440,192.543933,8.399372,244.133891,NaN,43.018828,316.560669,226.196653,NaN,NaN
std,NaN,NaN,NaN,NaN,5.185954,8.044202,583.746313,0.464843,686.372431,NaN,89.704486,1292.392383,499.563662,NaN,NaN
min,NaN,NaN,NaN,NaN,41.387399,-0.129001,1.000000,7.100000,2.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,47.377109,2.346964,22.250000,8.100000,29.250000,NaN,6.000000,24.000000,15.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,48.850984,12.494443,56.500000,8.400000,76.000000,NaN,15.000000,57.500000,53.500000,NaN,NaN
75%,NaN,NaN,NaN,NaN,51.509757,16.366150,145.250000,8.700000,192.500000,NaN,42.000000,174.000000,181.000000,NaN,NaN


## Dataprep.EDA

In [11]:
# Run me if you'd like to install dataprep
# !pip install dataprep
from dataprep.eda import plot as edaplot
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing

NumExpr defaulting to 8 threads.


In [12]:
edaplot(fs_df)

In [13]:
plot_missing(fs_df)

### Is there any relation between the rating and the number of photos?

In [14]:
plot_correlation(fs_df)

## Number of places in each category

In [15]:
fs_df.groupby(['category']).size()

category
African Restaurant        1
Argentinian Restaurant    1
Art Gallery               5
Art Museum                3
Arts & Crafts Store       2
                         ..
Whisky Bar                1
Wine Bar                  9
Wine Shop                 3
Yoga Studio               1
Zoo                       1
Length: 156, dtype: int64

## Top 20 Categories

In [16]:
plt1 = fs_df.groupby(['category']).name.count().sort_values(ascending=False).head(20)
plt1

category
Hotel                  36
Café                   20
Italian Restaurant     16
Plaza                  16
Coffee Shop            12
Bakery                 11
French Restaurant      11
Cocktail Bar           11
Park                   11
Ice Cream Shop         11
Restaurant             10
Bar                    10
Pub                    10
Wine Bar                9
Theater                 8
Burger Joint            8
Japanese Restaurant     8
Spanish Restaurant      7
Bistro                  7
Chocolate Shop          5
Name: name, dtype: int64

In [17]:
trace = go.Bar(
    x=plt1.index,
    y=plt1.values,
    marker=dict(
        color = plt1.values,
        colorscale='Jet',
        showscale=True
    )
)

data = [trace]
layout = go.Layout(
    title='Top 20 recomended Categories', 
    yaxis = dict(title = '# of Flights')
)

fig = go.Figure(data=data, layout=layout)
#for showing on the notbook
py.iplot(fig)

## Top 20 categories recomended by cities

In [18]:
plt2 = fs_df.groupby(['category', 'city']).name.count().sort_values(
    ascending=False).head(20).unstack().T
plt2.plot(kind='bar')

## Most liked places by categories in different cities

In [19]:
plt3 = fs_df.groupby(['city', 'category',
                      'name']).likes.max().sort_values(ascending=False).head(
                          20).unstack().stack() #most liked in all
plt3

city       category            name                                                             
Paris      Church              Cathedral of Notre-Dame de Paris (Cathédrale Notre-Dame de Paris)    8626.0
           Plaza               Place Saint-Michel                                                    753.0
           Bookstore           Shakespeare & Company                                                1420.0
London     Art Museum          National Gallery                                                     4691.0
           Plaza               Trafalgar Square                                                     4530.0
           Art Gallery         National Portrait Gallery                                            1522.0
           Wine Bar            Gordon's Wine Bar                                                    1271.0
           Mexican Restaurant  Wahaca                                                                811.0
           Sushi Restaurant    Sticks'n'Sushi  

## most liked places in each city

In [20]:
#max likes in each city
idx = fs_df.groupby(['city'])['likes'].transform(max) == fs_df['likes']
map1_df = fs_df[idx]
map1_df[['name','city','category','lat','lon','likes']]

,name,city,category,lat,lon,likes
10,Linnanmäki,Helsinki,Theme Park,60.188179,24.940485,710
17,Döner Harju,Harju,Doner Restaurant,60.188743,24.953261,346
50,Cathedral of Notre-Dame de Paris (Cathédrale N...,Paris,Church,48.853124,2.349561,8626
116,Vltava Riverside (Pražská náplavka),Praha,Waterfront,50.071242,14.413921,1516
150,National Gallery,London,Art Museum,51.508876,-0.128478,4691
194,J Sheekey,Charing Cross,Seafood Restaurant,51.510899,-0.127797,269
198,Franco Manca,Covent Garden,Pizza Place,51.510909,-0.122780,305
201,Paco Meralgo,Barcelona,Tapas Restaurant,41.391676,2.152794,536
296,Basilica di Santa Maria Maggiore,Roma,Church,41.897629,12.498429,1345
317,Amsterdam Central Railway Station (Station Ams...,Amsterdam,Train Station,52.378782,4.900374,3849


In [21]:
eu_map1 = folium.Map(
    location=[60.192059, 24.945831],
    zoom_start=3,
    max_zoom=6,
    min_zoom=2,
    prefer_canvas=True)  

# add the Italian restaurants as blue circle markers
for lat, lon, label, likes in zip(map1_df.lat, map1_df.lon, map1_df.name,
                                   map1_df.likes):
    folium.features.CircleMarker([lat, lon],
                                 radius=5,
                                 color='blue',
                                 fill=True,
                                 fill_color='blue',
                                 fill_opacity=0.6,
                                 popup=label + '<br>' + 'likes' +
                                 str(likes)).add_to(eu_map1)

# display map
eu_map1

## Value counts of the ratings

In [22]:
fs_df['rating'].value_counts().sort_values(ascending=False).plot(kind='bar')

## Highest rated places (rating counts taken into account) in each city

In [23]:
#max rating
idx = fs_df.groupby(['city'])['rating'].transform(max) == fs_df['rating']
map2_df = fs_df[idx]
map2_df[['name','city','category','lat','lon','rating']]

,name,city,category,lat,lon,rating
17,Döner Harju,Harju,Doner Restaurant,60.188743,24.953261,8.3
31,Riviera,Helsinki,Indie Movie Theater,60.187863,24.957700,9.5
50,Cathedral of Notre-Dame de Paris (Cathédrale N...,Paris,Church,48.853124,2.349561,9.4
73,La Maison d'Isabelle,Paris,Bakery,48.850007,2.348443,9.4
137,Invisible Exhibition (Neviditelná výstava),Praha,Exhibit,50.078432,14.421187,9.3
144,Papelote,Praha,Paper / Office Supplies Store,50.078331,14.414984,9.3
150,National Gallery,London,Art Museum,51.508876,-0.128478,9.3
156,Corinthia Hotel,London,Hotel,51.506607,-0.124460,9.3
173,Amorino,Charing Cross,Ice Cream Shop,51.510427,-0.128401,8.9
198,Franco Manca,Covent Garden,Pizza Place,51.510909,-0.122780,8.7


In [24]:
eu_map2 = folium.Map(
    location=[60.192059, 24.945831],
    zoom_start=3,
    max_zoom=6,
    min_zoom=2,
    prefer_canvas=True)  

# add the Italian restaurants as blue circle markers
for lat, lon, label, rating in zip(map2_df.lat, map2_df.lon, map2_df.name,
                                   map2_df.rating):
    folium.features.CircleMarker([lat, lon],
                                 radius=5,
                                 color='blue',
                                 fill=True,
                                 fill_color='blue',
                                 fill_opacity=0.6,
                                 popup=label + '<br>' + 'rating' +
                                 str(rating)).add_to(eu_map2)

# display map
eu_map2

## Minimum rated places (rating counts taken into account) in each city

In [25]:
#min rating
idx = fs_df.groupby(['city'])['rating'].transform(min) == fs_df['rating']
map3_df = fs_df[idx]
map3_df[['name','city','category','rating','lat','lon']]

,name,city,category,rating,lat,lon
17,Döner Harju,Harju,Doner Restaurant,8.3,60.188743,24.953261
36,Thai Orchid,Helsinki,Thai Restaurant,7.1,60.193210,24.943615
45,Kahvihuone,Helsinki,Vietnamese Restaurant,7.1,60.190816,24.948483
78,Point Zéro des Routes de France,Paris,Historic Site,7.8,48.853378,2.348827
95,Rasa Yoga,Paris,Yoga Studio,7.8,48.851454,2.346311
132,Lelí's Cupcakes,Praha,Cupcake Shop,7.4,50.073333,14.418451
139,Hotel U Svatého Jana,Praha,Hotel,7.4,50.071967,14.418435
145,Freshland,Praha,Salad Place,7.4,50.074523,14.418084
167,Bancone,Covent Garden,Italian Restaurant,8.4,51.509529,-0.126434
180,Waterstones,London,Bookstore,8.1,51.507650,-0.127130


In [26]:
eu_map3 = folium.Map(
    location=[60.192059, 24.945831],
    zoom_start=3,
    #max_zoom=6,
    #min_zoom=2,
    prefer_canvas=True)  

# add the Italian restaurants as blue circle markers
for lat, lon, label, rating in zip(map3_df.lat, map3_df.lon, map3_df.name,
                                   map3_df.rating):
    folium.features.CircleMarker([lat, lon],
                                 radius=5,
                                 color='blue',
                                 fill=True,
                                 fill_color='blue',
                                 fill_opacity=0.6,
                                 popup=label + '<br>' + 'rating' +
                                 str(rating)).add_to(eu_map3)

# display map
eu_map3

## Maximum weighted rating 

In [27]:
fs_df['weighted_rating'] = fs_df['rating'] * fs_df['rating_count']

In [28]:
#max weighted rating
idx = fs_df.groupby(['city'])['weighted_rating'].transform(max) == fs_df['weighted_rating']
map4_df=fs_df[idx]
map4_df[['name','city','category','weighted_rating','lat','lon']]
map4_df.shape

,name,city,category,weighted_rating,lat,lon
10,Linnanmäki,Helsinki,Theme Park,7550.8,60.188179,24.940485
17,Döner Harju,Harju,Doner Restaurant,3710.1,60.188743,24.953261
50,Cathedral of Notre-Dame de Paris (Cathédrale N...,Paris,Church,91565.4,48.853124,2.349561
116,Vltava Riverside (Pražská náplavka),Praha,Waterfront,15805.6,50.071242,14.413921
150,National Gallery,London,Art Museum,49373.7,51.508876,-0.128478
194,J Sheekey,Charing Cross,Seafood Restaurant,3255.5,51.510899,-0.127797
198,Franco Manca,Covent Garden,Pizza Place,3506.1,51.510909,-0.122780
201,Paco Meralgo,Barcelona,Tapas Restaurant,6828.8,41.391676,2.152794
296,Basilica di Santa Maria Maggiore,Roma,Church,14147.0,41.897629,12.498429
317,Amsterdam Central Railway Station (Station Ams...,Amsterdam,Train Station,41450.2,52.378782,4.900374


(14, 16)

In [29]:
eu_map4 = folium.Map(
    location=[60.192059, 24.945831],
    zoom_start=3,
   # max_zoom=6,
   # min_zoom=2,
    prefer_canvas=True) 

# add the Italian restaurants as blue circle markers
for lat, lon, label, rating in zip(map4_df.lat, map4_df.lon, map4_df.name,
                                   map4_df.weighted_rating):
    folium.features.CircleMarker([lat, lon],
                                 radius=5,
                                 color='blue',
                                 fill=True,
                                 fill_color='blue',
                                 fill_opacity=0.6,
                                 popup=label + '<br>' + 'Weighted_rating' +
                                 str(rating)).add_to(eu_map4)

# display map
eu_map4

## Maximum tips from each city

In [30]:
idx = fs_df.groupby(['city'])['tip_count'].transform(max) == fs_df['tip_count']
map5_df=fs_df[idx]
map5_df[['name','city','category','tip_count','lat','lon']]

,name,city,category,tip_count,lat,lon
10,Linnanmäki,Helsinki,Theme Park,89,60.188179,24.940485
17,Döner Harju,Harju,Doner Restaurant,58,60.188743,24.953261
50,Cathedral of Notre-Dame de Paris (Cathédrale N...,Paris,Church,978,48.853124,2.349561
123,Café Amandine,Praha,Café,172,50.073770,14.417530
151,Trafalgar Square,London,Plaza,553,51.507987,-0.128048
194,J Sheekey,Charing Cross,Seafood Restaurant,110,51.510899,-0.127797
198,Franco Manca,Covent Garden,Pizza Place,67,51.510909,-0.122780
201,Paco Meralgo,Barcelona,Tapas Restaurant,236,41.391676,2.152794
295,La Romana,Roma,Ice Cream Shop,336,41.907725,12.498984
317,Amsterdam Central Railway Station (Station Ams...,Amsterdam,Train Station,681,52.378782,4.900374
